In [1]:
# ## install finrl library
# !pip install wrds
# !pip install quantstats
# !pip install torch_geometric
# !pip install swig
# !pip install -q condacolab
# !pip install shimmy
# import condacolab
# condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
# !pip install git+https://github.com/flpymonkey/FinRL_Online_Portfolio_Benchmarks.git

In [2]:
import pandas as pd
from stable_baselines3.common.logger import configure


from finrl.config import TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.main import check_and_make_directories

import numpy as np


TIME_WINDOW = 25
COMMISSION_FEE_PERCENT = 0.001
INITIAL_CASH = 1000000


TRAIN_START_DATE = '2009-04-01'
TRAIN_END_DATE = '2021-12-31'
TEST_START_DATE = '2022-01-01'
TEST_END_DATE = '2024-09-01'


TEST_TICKER = [
   "MSFT",
    "V",
    "AAPL",
    "BA",
    "INTC",
    "WMT",
]

TRAINED_PPO = "/agent_opt_ppo_update_test"

GRAPH_TITLE = "PPO Trained 2009-2021, test softmax"



from finrl.meta.preprocessor.yahoodownloader import YahooDownloader



df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = TEST_TICKER).fetch_data()

processed = df.copy()
processed = processed.fillna(0)
processed_test = processed.replace(np.inf,0)


from sklearn.preprocessing import MaxAbsScaler
from finrl.meta.preprocessor.preprocessors import GroupByScaler

portfolio_norm_df = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(processed_test)
portfolio_norm_df

df_train = portfolio_norm_df[(portfolio_norm_df["date"] >= TRAIN_START_DATE) & (portfolio_norm_df["date"] <= TRAIN_END_DATE)]
df_test = portfolio_norm_df[(portfolio_norm_df["date"] >= TEST_START_DATE) & (portfolio_norm_df["date"] < TEST_END_DATE)]

 

df_train.groupby("tic").count()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (23286, 8)


c:\Users\bencj\Desktop\Econ4\Thesis\.venv\lib\site-packages\finrl\meta\preprocessor\preprocessors.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.31332941 0.43188074 0.33824661 ... 0.02022979 0.02759509 0.02817164]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[select_mask, self.columns] = self.scalers[value].transform(
c:\Users\bencj\Desktop\Econ4\Thesis\.venv\lib\site-packages\finrl\meta\preprocessor\preprocessors.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.5  0.75 1.   ... 0.5  0.75 1.  ]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  X.loc[select_mask, self.columns] = self.scalers[value].transform(


,date,open,high,low,close,volume,day
tic,,,,,,,
AAPL,3212,3212,3212,3212,3212,3212,3212
BA,3212,3212,3212,3212,3212,3212,3212
INTC,3212,3212,3212,3212,3212,3212,3212
MSFT,3212,3212,3212,3212,3212,3212,3212
V,3212,3212,3212,3212,3212,3212,3212
WMT,3212,3212,3212,3212,3212,3212,3212


In [3]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [4]:
# Fit for the portfolio optimization model

from finrl.meta.env_portfolio_optimization.env_portfolio_optimization import PortfolioOptimizationEnv


from sklearn.preprocessing import MaxAbsScaler
from finrl.meta.preprocessor.preprocessors import GroupByScaler

from finrl.meta.preprocessor.preprocessors import data_split

from datetime import datetime, timedelta


environment_ppo = PortfolioOptimizationEnv(
    df_train,
    initial_amount=INITIAL_CASH,
    comission_fee_pct=COMMISSION_FEE_PERCENT,
    # time_window=TIME_WINDOW,
    features=["close", "high", "low"],
    normalize_df=None,
    reward_scaling=1e-4
)

In [5]:
from finrl.agents.portfolio_optimization.models_stable import DRLStableAgent



agent_ppo = DRLStableAgent(env = environment_ppo)
PPO_PARAMS = {
    "n_steps": 2048,
    "batch_size": 64,
    # "ent_coef": 0.01,
    "learning_rate": 0.00025, # tried raising the lr which caused vanishing problem
    "clip_range": 0.2,
    # "gae_lambda": 0.001,
}

# PPO_policy_params = { 
#     "log_std_init": 1
# }

# Lower clip_range makes the stocks flatter, very conservative policy

# TODO try playing around with the number of epochs? n_epochs
# TODO try playing around more with the entropy term, make sure agent does enough exploration during training
# TODO try playing around more with the clip param here


model_ppo = agent_ppo.get_model("ppo", device, model_kwargs=PPO_PARAMS, policy_kwargs=None)

# set up logger
tmp_path = RESULTS_DIR + '/ppo'
new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
# Set new logger
model_ppo.set_logger(new_logger_ppo)

c:\Users\bencj\Desktop\Econ4\Thesis\.venv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Logging to results/ppo


In [6]:
model_ppo = DRLStableAgent.train_model(model_ppo, env=environment_ppo, episodes=1)

from finrl.config import TRAINED_MODEL_DIR

environment_ppo.reset()

model_ppo.save(TRAINED_MODEL_DIR + "/agent_opt_ppo_update_test")

Output()

Max number of time steps in an episode:  3212


-----------------------------
| time/              |      |
|    fps             | 94   |
|    iterations      | 1    |
|    time_elapsed    | 21   |
|    total_timesteps | 2048 |
-----------------------------


=================================

Initial portfolio value:1000000

...

Maximum DrawDown: -0.5162372864356897

Sharpe ratio: 0.0005692855919064612

Total commission cost: 2696375.5511937547

=================================

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 3.21e+03  |
|    ep_rew_mean          | -2.2e-05  |
| time/                   |           |
|    fps                  | 82        |
|    iterations           | 2         |
|    time_elapsed         | 49        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 12.799808 |
|    clip_fraction        | 0.961     |
|    clip_range           | 0.2       |
|    entropy_loss         | -10       |
|    explained_variance   | 0.916     |
|    learning_rate        | 0.00025   |
|    loss                 | 0.519     |
|    n_updates            | 10        |
|    policy_gradient_loss | 28.5      |
|    std                  | 1.01      |
|    value_loss           | 0.000138  |
---------------------------------------


In [7]:
from stable_baselines3 import PPO
from finrl.config import TRAINED_MODEL_DIR


# Load the trained models
# trained_ppo_opt = PPO.load(TRAINED_MODEL_DIR + "/agent_opt_ppo_10_27") 




trained_ppo_opt = PPO.load(TRAINED_MODEL_DIR + TRAINED_PPO) 


# trained_ppo_opt = model_ppo


In [8]:
agent_ppo_test = DRLStableAgent(env = environment_ppo)

PPO_results = {
    "train": {},
}

values, dates = DRLStableAgent.DRL_prediction(trained_ppo_opt, environment_ppo, verbose=True)
PPO_results["train"]["value"] = environment_ppo._terminal_asset_memory["final"]
PPO_results["train"]["date"] = environment_ppo._terminal_date_memory


# Write this out to a csv file, with date and net worth
df_ppo_opt = pd.DataFrame(PPO_results["train"]["value"], columns=['ppo_opt_net_worth'])
df_ppo_date = pd.DataFrame(PPO_results["train"]["date"], columns=['Date'])
if len(df_ppo_opt) == len(df_ppo_date):
    df_ppo_opt['Date'] = df_ppo_date['Date']
else:
    raise ValueError("DataFrames do not have the same number of rows.")

print(df_ppo_opt)


print(df_ppo_opt.loc[0, 'Date'])


c:\Users\bencj\Desktop\Econ4\Thesis\.venv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Step:  0
Observations: 
[[[[0.0139761 ]
   [0.06240007]
   [0.15314019]
   [0.0307046 ]
   [0.0420142 ]
   [0.16277684]]

  [[0.01640963]
   [0.07981883]
   [0.22109972]
   [0.04133661]
   [0.04811658]
   [0.22826038]]

  [[0.01591813]
   [0.07771644]
   [0.21720399]
   [0.03914223]
   [0.04656999]
   [0.2240007 ]]]]
Actions: 
[[0.14242876 0.14339912 0.14211372 0.14369446 0.14244798 0.14351198
  0.14240396]]
Step:  1
Observations: 
[[[[0.01449301]
   [0.06549895]
   [0.1599668 ]
   [0.03067279]
   [0.04302026]
   [0.16530383]]

  [[0.01727528]
   [0.08515504]
   [0.23091355]
   [0.04246824]
   [0.04907032]
   [0.23480058]]

  [[0.0168206 ]
   [0.08214635]
   [0.22671224]
   [0.04090772]
   [0.04786384]
   [0.23265985]]]]
Actions: 
[[0.14239055 0.14343022 0.14205919 0.14377686 0.14242253 0.14357583
  0.1423448 ]]
Step:  2
Observations: 
[[[[0.01491478]
   [0.06636169]
   [0.16251405]
   [0.02981414]
   [0.046495  ]
   [0.16579695]]

  [[0.01748303]
   [0.08546893]
   [0.23033626]
   [0.

In [9]:
environment_ppo_test = PortfolioOptimizationEnv(
    df_test,
    initial_amount=INITIAL_CASH,
    comission_fee_pct=COMMISSION_FEE_PERCENT,
    # time_window=TIME_WINDOW,
    features=["close", "high", "low"],
    normalize_df=None,
    reward_scaling=1e-4
)

PPO_results = {
    "test": {},
}

values, dates = DRLStableAgent.DRL_prediction(trained_ppo_opt, environment_ppo_test, verbose=False)
PPO_results["test"]["value"] = environment_ppo_test._terminal_asset_memory["final"]
PPO_results["test"]["date"] = environment_ppo_test._terminal_date_memory


# Write this out to a csv file, with date and net worth
df_ppo_opt = pd.DataFrame(PPO_results["test"]["value"], columns=['ppo_opt_net_worth'])
df_ppo_date = pd.DataFrame(PPO_results["test"]["date"], columns=['Date'])
if len(df_ppo_opt) == len(df_ppo_date):
    df_ppo_opt['Date'] = df_ppo_date['Date']
else:
    raise ValueError("DataFrames do not have the same number of rows.")

print(df_ppo_opt)


print(df_ppo_opt.loc[0, 'Date'])

c:\Users\bencj\Desktop\Econ4\Thesis\.venv\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Initial portfolio value:1000000
Final portfolio value: 1091203.75
Final accumulative portfolio value: 1.09120375
Maximum DrawDown: -0.25742656249999996
Sharpe ratio: 0.27130545264531913
Total commission cost: 5496.3002194727305
hit end!
     ppo_opt_net_worth       Date
0         1.000000e+06 2022-01-03
1         9.976073e+05 2022-01-04
2         9.902381e+05 2022-01-05
3         9.854832e+05 2022-01-06
4         9.860020e+05 2022-01-07
..                 ...        ...
664       1.065301e+06 2024-08-26
665       1.066879e+06 2024-08-27
666       1.057854e+06 2024-08-28
667       1.070447e+06 2024-08-29
668       1.091204e+06 2024-08-30

[669 rows x 2 columns]
2022-01-03 00:00:00
